In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Run

In [1]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.runner as runner
import source.util as util

import source.util as util
import numpy as np
import pandas as pd
from importlib import *
import source.util as util
from copy import deepcopy, copy
from contextlib import redirect_stdout
import time 
import os
import re
import concurrent.futures
import logging
import sys
import datetime

### set logger

In [6]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
# run to change level

lvl = logging.DEBUG
logger.setLevel(lvl)
ch.setLevel(lvl)

In [14]:
reload(runner)

<module 'source.runner' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/source/runner.py'>

In [2]:
experiment_folder = "../NewExperiments/"

## parse batches

parse the batches in *experiment_folder* and put them in the multidimensional dict *bdict*

In [7]:
batches = []
bdict = {}
csv_pattern = re.compile("^.+\.csv$")
for root, dirs, files in os.walk(experiment_folder):
    if root != experiment_folder:
        args = root.split(experiment_folder)[-1].split("/")
        if not files:
            util.get_el(bdict, args[:-1])[args[-1]] = {}
        else:
            util.get_el(bdict, args[:-1])[args[-1]] = []
            for f in files:
                if (csv_pattern.match(f) and f != "batch.csv" and
                    f != "stats.csv" and f.split('.csv')[0] not in dirs):
                    #print(f)
                    b = runner.Batch(os.path.join(root, f), root)
                    b.parse_batch()
                    batches.append(b)
                    util.get_el(bdict, args).append(b)

In [8]:
len(batches)

1

## run batches in parallel

In [9]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=2)

## collect results

In [10]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, save=True, show=False, semilog=True,
                            path=b.results_folder_path,
                            title=confs[0].game.players[1].__class__.name)

## plot

In [11]:
for t in bdict:
    for p in bdict[t]:
        confs = [c for i in bdict[t][p]
                   for b in bdict[t][p][i]
                   for c in b.configurations]
        cdict = {}
        for c in confs:
            cname = c.name.split(" vs")[0]
            if cname not in cdict:
                cdict[cname] = [c]
            else:
                cdict[cname].append(c)
        dlist = []
        for n in cdict:
            avg_regrets = [c.stats["exp_regret"] for c in cdict[n]]
            d = util.avg_with_conf(avg_regrets, name=n)
            dlist.append(d)
        util.plot_dicts(dlist, save=True, show=False, semilog=False,
                path=os.path.join(experiment_folder, t, p),
                title=p)

## Experiments

verify what happened if something went wrong

In [6]:
g = deepcopy(batches[0].configurations[0].game)

In [7]:
e = runner.Experiment(g)

In [8]:
for i in range(900):
    e.run_interaction()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    print(list(e.agent.belief.pr.items())[2])

{0: [0.0, 1.0, 0.0, 0.0, 0.0], 1: [0.382617382617, 0.0779220779221, 0.0509490509491, 0.448551448551, 0.03996003996]}
{0: [1], 1: [0]}
(<StackelbergAttacker id:1 resources:1>, 0.0)
{0: [0.19914686880847135, 0.33207178953413297, 0.22270967465717414, 0.13930409348215264, 0.10676757351806875], 1: [0.382617382617, 0.0779220779221, 0.0509490509491, 0.448551448551, 0.03996003996]}
{0: [0], 1: [3]}
(<StackelbergAttacker id:1 resources:1>, 0.0)
{0: (0, 0, 0, 1, 0), 1: [0.382617382617, 0.0779220779221, 0.0509490509491, 0.448551448551, 0.03996003996]}
{0: [3], 1: [0]}
(<StackelbergAttacker id:1 resources:1>, 0.0)
{0: [0.19914686880847135, 0.33207178953413297, 0.22270967465717414, 0.13930409348215264, 0.10676757351806875], 1: [0.382617382617, 0.0779220779221, 0.0509490509491, 0.448551448551, 0.03996003996]}
{0: [1], 1: [1]}
(<StackelbergAttacker id:1 resources:1>, 0.0)
{0: (0, 0, 0, 1, 0), 1: [0.382617382617, 0.0779220779221, 0.0509490509491, 0.448551448551, 0.03996003996]}
{0: [3], 1: [2]}
(<Stac

In [44]:
e.run()